# MACHINE LEARNING PROJECT


## Importing packages

In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

## Loading dataset


In [4]:
def load_data(file_pattern, delimiter=';'):
    files = glob.glob(file_pattern)
    dfs = [pd.read_csv(file, delimiter=delimiter) for file in files]
    return pd.concat(dfs, ignore_index=True)

In [5]:
# Load training data for each neural network
efficientnet_train_data = load_data('EfficientNet/TRAIN_*.csv')
mobilenet_train_data = load_data('MobileNet/TRAIN_*.csv')

In [6]:
resnet_train_data = load_data('ResNet/TRAIN_*.csv')

In [7]:
efficientnet_test_data = pd.read_csv('efficientNet_test/TEST.csv', delimiter=';')
mobilenet_test_data = pd.read_csv('mobileNet_test/TEST.csv', delimiter=';')

In [8]:
resnet_test_data = pd.read_csv('resNet_test/TEST.csv', delimiter=';')

### Combining dataset

In [9]:
train_data_MobEff = pd.concat([efficientnet_train_data, mobilenet_train_data], ignore_index=True)
test_data_MobEff = pd.concat([efficientnet_test_data, mobilenet_test_data], ignore_index=True)

In [10]:
# Check if normalization is needed
#scaler = StandardScaler()
#scaler.fit(train_data_MobEff)
#X = scaler.transform(X)
#X_train = pd.DataFrame(X)

In [13]:
# Separate features and labels
X_train_MobEff = train_data_MobEff.drop(['images', 'cone_name', 'label'], axis=1)
y_train_MobEff = train_data_MobEff['label']
X_test_img_MobEff = test_data_MobEff.drop('image', axis=1)
y_test_img_MobEff = test_data_MobEff['image']

In [14]:
# Dimensionality reduction using PCA
pca = PCA(n_components=512)
X_train_pca = pca.fit_transform(X_train_MobEff)
X_test_pca = pca.transform(X_test_img_MobEff)

In [17]:
X_train = pd.DataFrame(X_train_pca)
X_test = pd.DataFrame(X_test_pca)

In [22]:
X_train.describe()


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
count,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,...,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04,2.234000e+04
mean,2.084428e-14,3.256919e-16,1.860642e-17,3.625867e-17,5.168451e-19,2.910235e-17,-4.691363e-18,2.798915e-17,-4.770878e-19,-2.210507e-17,...,-2.522602e-17,-1.035181e-16,4.002568e-17,-4.552213e-17,7.339200e-17,6.152445e-17,-5.456691e-17,2.163792e-17,-1.749322e-17,4.057234e-17
std,4.726595e+02,6.819472e+00,1.986144e-01,1.848435e-01,1.475377e-01,1.370622e-01,1.273030e-01,1.220188e-01,1.139078e-01,1.100807e-01,...,4.513269e-02,4.507469e-02,4.501939e-02,4.499931e-02,4.493739e-02,4.485689e-02,4.475442e-02,4.471558e-02,4.468450e-02,4.462253e-02
min,-5.014074e+02,-7.558698e+00,-7.389191e-01,-8.922880e-01,-7.198181e-01,-7.082027e-01,-6.629803e-01,-5.977716e-01,-6.210721e-01,-6.025254e-01,...,-2.084705e-01,-2.128160e-01,-2.317499e-01,-2.434982e-01,-2.250242e-01,-2.162186e-01,-2.200304e-01,-2.370004e-01,-2.195896e-01,-2.456966e-01
25%,-3.434070e+02,-6.849440e+00,-6.097126e-02,-5.657231e-02,-3.794031e-02,-3.566030e-02,-3.415963e-02,-3.788780e-02,-3.664784e-02,-3.568432e-02,...,-2.469207e-02,-2.469681e-02,-2.472003e-02,-2.463388e-02,-2.447737e-02,-2.453955e-02,-2.460733e-02,-2.448156e-02,-2.467766e-02,-2.433008e-02
50%,-1.524073e+02,5.389627e-03,-4.105438e-03,-6.800596e-03,2.701391e-04,5.548029e-04,1.356636e-03,-1.159823e-03,2.162010e-04,-5.353437e-04,...,3.258523e-05,-8.844323e-05,-1.541394e-04,-5.892624e-05,1.394725e-05,3.304091e-04,-2.105838e-05,1.465746e-04,-5.044146e-05,1.160892e-04
75%,1.845930e+02,6.807898e+00,3.866608e-02,3.797614e-02,3.957753e-02,3.668616e-02,3.776097e-02,3.449670e-02,3.601539e-02,3.344130e-02,...,2.410477e-02,2.455918e-02,2.476568e-02,2.441019e-02,2.444865e-02,2.423236e-02,2.442038e-02,2.414994e-02,2.479541e-02,2.457561e-02
max,1.629593e+03,8.330602e+00,1.011969e+00,1.141803e+00,7.964282e-01,8.229147e-01,8.386627e-01,6.956426e-01,6.677785e-01,6.058978e-01,...,2.592981e-01,2.190968e-01,2.648503e-01,2.455793e-01,2.609009e-01,2.253486e-01,2.557136e-01,2.385569e-01,2.511151e-01,2.574063e-01


In [23]:
X_train.head()


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-501.407314,6.750191,0.261442,-0.178307,0.415324,0.059685,-0.196913,-0.155857,-0.273663,-0.195214,...,-0.025043,-0.031105,-0.080198,-0.044606,0.045440,0.103556,-0.014049,0.084093,0.125191,0.067740
1,-500.407307,6.499138,0.447432,-0.055049,0.426733,0.210695,0.058464,-0.244583,-0.033781,-0.208856,...,0.007772,-0.095620,-0.017727,-0.011252,-0.005105,0.019970,-0.043972,-0.028299,-0.065463,-0.012170
2,-499.407314,6.507906,0.302485,-0.029843,-0.087973,0.036247,-0.056748,-0.161522,0.097432,0.118639,...,0.006192,0.119590,-0.046339,-0.023605,-0.067304,-0.010252,0.041325,-0.005555,-0.051045,-0.086070
3,-498.407330,6.868513,0.070631,0.080513,0.018756,-0.168464,-0.057058,0.038742,0.393939,0.261218,...,-0.052037,-0.034289,-0.019240,-0.016744,-0.089725,0.024753,0.004844,-0.012377,-0.108944,0.051349
4,-497.407313,6.360386,0.654985,0.412893,0.025627,-0.315959,-0.150268,-0.043862,0.287801,0.061796,...,0.045611,-0.004768,0.021374,0.033242,-0.080904,0.020504,0.000009,0.065536,-0.044918,0.059617


In [10]:
# Saving data to csv to confirm the proper concatination of data
#train_data.to_csv('combined_train_data.csv', index=False)
#test_data.to_csv('combined_test_data.csv', index=False)

## Pre-processing & Normalization

In [11]:
# Dropping the first column 
# train_data.drop('Unnamed', axis=1)

**_X_train_** is dataset without last three columns

**_y_train_** contains only **_labels_** column

**_X_test_img_** is the test dataset without **_image_** column

**_y_test_img_** contains only **_image_** column

In [ ]:
# Separate features and labels
X_train_MobEff = train_data_MobEff.drop(['images', 'cone_name', 'label'], axis=1)
y_train_MobEff = train_data_MobEff['label']
X_test_img_MobEff = test_data_MobEff.drop('image', axis=1)
y_test_img_MobEff = test_data_MobEff['image']

In [13]:
#from sklearn.impute import SimpleImputer

#imputer = SimpleImputer(strategy='mean')
#X_train_imputed = imputer.fit_transform(X_train)
#
# Ensure imputation has been done correctly
#X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)

Large portion of dataset is missing values, imputation might not be reliable.

Consider models that can handle missing values inherently, such as tree-based models

Some models inherently handle missing values better than others. For example, Decision Trees 

and Random Forests can handle missing values by making decisions based on available data, 

while linear models and Naive Bayes typically require complete data.

In [14]:
#X_train = X_train_imputed

In [15]:
#X_train.head()

In [16]:
#X_test_img.head()

In [17]:
#y_train.head()

In [18]:
#y_test_img.head()

In [19]:
#X_train.describe()

In [20]:
# Don't drop any row because 

#X_train.dropna(inplace=True)
#X_train.describe()

## PCA Dimensionality Reduction

## Train-Test splitting

Splitting **_train_** dataset into **70/30** ratio

**_X_train_** is dataset without last three columns

**_y_train_** contains only **_labels_** column

**_X_test_img_** is the test dataset without **_image_** column

**_y_test_img_** contains only **_image_** column

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [24]:
#X_train.describe()

# Gradient Boosting



In [25]:
y_train = y_train - 1
y_test = y_test - 1

In [26]:
#from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=400, random_state=42)

# Train the model
start_time = time.time()
xgb_model.fit(X_train, y_train)
end_time = time.time()

train_time = end_time-start_time
#print(f'Train time: {train_time}')

In [27]:
# Make predictions
#y_pred_xgb = xgb_model.predict(X_test)

# Evaluate accuracy
#accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
#print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

In [28]:
"""# Define the parameter grid
param_grid = {
    'n_estimators': [400]
}

# Initialize the XGBClassifier
xgb = XGBClassifier(random_state=42)

# Initialize GridSearchCV
start_time = time.time()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

end_time = time.time()

train_time = end_time-start_time
print(f'Train time: {train_time}')
"""

"# Define the parameter grid\nparam_grid = {\n    'n_estimators': [400]\n}\n\n# Initialize the XGBClassifier\nxgb = XGBClassifier(random_state=42)\n\n# Initialize GridSearchCV\nstart_time = time.time()\ngrid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)\n\n# Fit GridSearchCV\ngrid_search.fit(X_train, y_train)\n\nend_time = time.time()\n\ntrain_time = end_time-start_time\nprint(f'Train time: {train_time}')\n"

In [29]:
"""# Best parameters and best score
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best cross-validation accuracy: {grid_search.best_score_:.2f}')

# Use the best model
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train, y_train)
y_pred_best_xgb = best_xgb.predict(X_test)
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
print(f'XGBoost Best Model Accuracy: {accuracy_best_xgb:.2f}')
"""

"# Best parameters and best score\nprint(f'Best parameters: {grid_search.best_params_}')\nprint(f'Best cross-validation accuracy: {grid_search.best_score_:.2f}')\n\n# Use the best model\nbest_xgb = grid_search.best_estimator_\nbest_xgb.fit(X_train, y_train)\ny_pred_best_xgb = best_xgb.predict(X_test)\naccuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)\nprint(f'XGBoost Best Model Accuracy: {accuracy_best_xgb:.2f}')\n"